# A simple model for estimating the time, cost and energy to train a frontier model

In the following, we aim to develop a simple model for estimating the time, cost and energy requirements of a large transformer model training run, given the model and dataset sizes, as well as some key information about the AI accelerators (GPUs) that are going to be used, such as their computing speed in FLOPs per second, memory, bandwidth interconnect speed and energy consumption. 

Our goal is not to calculate the above quantities precisely (as it could be challenging), but to find a reasonable estimate that is correct within a relatively small correction factor (less than an order of magnitude, ideally 50%). To achieve this, we’re going to do a series of simplifications:
- In the initial version, we’ll focus on Data Parallelism, and ignore Model and Tensor Parallelism and possible aspects thereof (delays due to model splitting across accelerators, latencies, etc.).
- We’ll focus on Transformer models, as it’s the predominant architecture for frontier models. According to Hoffmann et al. (https://arxiv.org/abs/2203.15556), each token requires approximately $6N$ parameters to perform the forward and backward pass, where $N$ is the number of model parameters.
- We’ll work with the 16 bit (2 byte) floating point format (`bfloat16`) for all numbers (model parameters and data tokens), as it’s the most typical in frontier model training. That is, each number takes 2 bytes to store.
Our model will use the following parameters for the model and dataset:
- $N$: the model size (number of parameters)
- $D$: the dataset size (number of tokens)

Additionally, we’ll assume the following parameters for the training cluster:
- $n$: number of AI accelerators (GPUs)
- $F$: GPU FLOPs per second (FLOP/s) in the `bfloat16` format
- $M$: GPU memory in bytes
- $B$: GPU interconnect speed (in bytes / sec). In the simplified version of our model we won’t distinguish between inter-server and intra-server bandwidth (which typically differ a bit), and take both to be the same as the GPUs maximum data transfer rate
- $W$: GPU power consumption (in Watt)



## Training time

Let’s start with the time required for training. A lower bound ($T_{flop}$) of it is the time required to process all tokens; since each token requires $6N$ FLOPs, and we have $D$ tokens in total, the total training FLOPs required are $6ND$; parallelizing this to $n$ AI accelerators, each with $F$ FLOP/s each, gives us a total time of:

$$
T_{flop} = \frac{6ND}{nF}
$$

Typically, only a fraction of the maximum possible FLOPs of a GPU is utilized; this is accounted for by the Mean FLOP Utilization factor, which for modern data center GPUs is estimated to be about $a_{MFU}=40\%$. This means that the actual FLOP/s we get can be estimated as $F=a_{MFU} F_{max}$, where $F_{max}$ is the maximum value specified by the manufacturer. 


However, this estimate ignores the communication time required after each model update (gradient step), where all GPUs need to synchronize with each other and update model parameters. 

At each gradient step, accelerators need to send the content of their memory to each other; this will require a time of $M/B$ seconds. It’s reasonable to assume that the memory of each device will be almost fully utilized (as we want to process as many tokens in parallel as possible), thus we estimate here that the gradient parameters in each device will take up almost the full memory.

Additionally, all n accelerators need to communicate their gradients with each other. The most efficient way to do this is with a tree-like structure, where at each step the number of synched devices doubles. Overall, this requires a communication time $t_{comm}$ after each model update that is equal to:

$$
t_{comm} = \frac{M}{B} \log_2(n)
$$

Assuming that each GPU can perform computations and communicate data in parallel, during a gradient update time $t_{comm}$, the GPUs can process a batch size $b$ of tokens, where $b$ can be found by the equation: $6Nb=nFt_{comm}$ (since this is the number of FLOPs possible to perform within time tcomm). Therefore, this limits the batch size to:

$$
b = \frac{n F t_{comm}}{6 N} = \frac{n F M \log_2(n)}{6 N B}
$$

Hence, the time needed to process the entire dataset of $D$ tokens would be: $T_{comm} = D b t_{comm} = 6 N D n F = T_{flop}$; e.g. in theory, communication latencies could be compensated by a large enough batch size ($b$ grows as $B$ decreases, and becomes arbitrarily large when $B$ is close to zero).



![GPU gradient all-reduce illustration](GPU_allReduce_fig.jpg)

Figure: Illustration of gradient communication between GPU devices ($4$ devices in this example). Initially, each GPU has a local copy of the gradient for its own micro-batch ($g_1, ..., g_4$). At step $1$, each accelerator synchronizes (averages) it’s gradient with its neighbor, resulting in accumulated gradients $g_{1,2}$, $g_{3,4}$ that are synchronized across “segments” of $2$ devices. Then at step $2$, devices $1$ and $3$ (and simultaneously in parallel devices $2$ and $4$) accumulate gradients $g_{1,2}$ and $g_{3,4}$ to produce the overall gradient $g_{1,2,3,4}$. A the number of synched devices doubles at each round, we see that the overall number of steps needed is $\log_2(n)$, where $n$ is the total number of AI accelerators.  

However, in practice this is not the case: it turns out that after some “critical batch size” $b_{max}$, increasing the batch size further yields diminishing and even zero returns (https://epochai.org/blog/can-ai-scaling-continue-through-2030, https://arxiv.org/abs/1812.06162). For GPT-4, the batch size is estimated to be around $6 \cdot 10^6$ tokens, and this value is regarded as close to the critical size for language modeling. Therefore, a more reasonable estimate is:
$$
b = \min(\frac{n F M \log_2(n)}{6 N B}, b_{max})
$$

Finally, the training time is estimated as:
$$
T = \frac{D}{b} t_{comm} = \frac{M D \log_2(n)}{B \min(\frac{n F M \log_2(n)}{6 N B}, b_{max})}
$$

This is our final training time estimation formula. 



## Energy consumption

For the energy consumption, we simply multiply the consumption in Watt per accelerator, by their number and training time. Additionally, we need to account for the fact that the total power consumption of the data center is does not come only from the GPUs, but also from the rest of the equipment, cooling etc. We can account for these by the power usage effectiveness (PUE) coefficient $a_{PUE}$, which is estimated around $a_{PUE} = 1.3$ for modern GPUs such as the NVIDIA H100 (https://arxiv.org/abs/2104.10350). Hence, the used energy is:

$$
E = n a_{PUE} W T
$$


## Cost estimation

The various training costs can then be estimated from these figures. For example, the electricity cost is the energy used divided by the cost per Joule ($c_J)$:

$$
C_{energy} = \frac{E}{c_J}
$$

For example, for industrial electricity in the US we have $c_J = 0.08 \$/kWh = 2.2210-8 \$/J$ (https://www.statista.com/statistics/1395805/monthly-electricity-price-industrial-sector-united-states/).

If the GPUs are rented from the cloud, the cost is calculated by the GPU cost rate per second crent; for example, for NVIDIA H100s, we have $c_{rent} = 2.5 \$/h = 6.910-4 \$/s$ for the cheapest option we found (https://lambdalabs.com/service/gpu-cloud). The total cost is then:

$$
C_{rent} = n T c_{rent}
$$

Finally, the hardware acquisition cost is:

$$
C_{hardware} = n P_{GPU} c_{adj}
$$
where $P_{GPU}$ is the price per GPU, and $c_{adj}$ is a factor to adjust for the additional data center and infrastructure costs, and can be approximated by $c_{adj} \approx 2$ as a rule of thumb (https://www.semianalysis.com/p/100000-h100-clusters-power-network).

With that, our model is complete, and we can proceed to analyze some simple examples.


## Implementation

In [6]:
import math

def training_estim(N, D, F_max, M, B, W, b_max, a_mfu, a_pue, c_j, c_rent, c_adj, P_gpu):
    # estimates the training time, energy and costs as analyzed above

    F = F_max * a_mfu # effective FLOP/s

    # training time calculation from section 1
    b = min(n * F * M * math.log2(n) / (6 * N * B), b_max)
    T = M * D * math.log2(n) / (B * b) 

    # energy calculation from section 2
    E = n * a_pue * W * T

    # cost estimations from section 3
    C_energy = E * c_j
    C_rent = n * T * c_rent
    C_hardware = n * P_gpu * c_adj

    # display the results
    print('Training time: {:.0f} days'.format(T / 3600 / 24))

    print('Energy used: {:.2E} kWh'.format(E / (1000 * 3600)))

    print('Energy cost: {:.2f} million $'.format(C_energy / 1e6))
    print('Renting cost: {:.2f} million $'.format(C_rent / 1e6))
    print('Hardware cost: {:.2f} million $'.format(C_hardware / 1e6))
# end func

In [7]:
# common parameters

# Max batch size
b_max = 60e6

# Factors
a_mfu = 0.4 # GPU utilization
a_pue = 1.3 # GPU energy efficiency

c_j = 2.22e-8
c_adj = 2

## Example 1: GPT-4

For GPT-4, researchers estimate a model size $N = 10^{12}$ parameters and $D = 13 \cdot 10^{12}$ tokens (https://the-decoder.com/gpt-4-architecture-datasets-costs-and-more-leaked/). The model was trained on $n = 10^4$ NVIDIA H100 equivalents, for around $90$ days. For the H100, we have $F = 1980 \cdot 10^{12}$ FLOP/s, $M = 80 \cdot 10^9$ bytes, $B = 900 \cdot 10^9$ bytes/s from the specs (https://www.nvidia.com/en-us/data-center/h100/). With that, our model gives:

In [8]:
# model data (GPT-4)
N = 1e12
D = 13e12

# cluster data (GPT-4)
n = 10000  # Number of H100 GPUs in the cluster

# Hardware parameters for NVIDIA H100 GPUs
F_max = 1980e12  # 1980 TFLOPS for NVIDIA H100 in FP16 precision
B = 900e9  # e.g., 900 GB/s for NVLink on H100
M = 80e9  # 80 GB for NVIDIA H100
W = 700 # GPU Watt consumption

# price (purchase and rent) for the NVIDIA H100
c_rent = 6.9e-4
P_gpu = 25e3

In [9]:
# get estimates
training_estim(N, D, F_max, M, B, W, b_max, a_mfu, a_pue, c_j, c_rent, c_adj, P_gpu)

Training time: 114 days
Energy used: 2.49E+07 kWh
Energy cost: 1.99 million $
Renting cost: 67.95 million $
Hardware cost: 500.00 million $


These values are close to the public estimates.

## Example 2: Llama 3.1 405b


For the largest Llama model, we have a size of $N = 405 \cdot 10^9$ and $D = 15.6 \cdot 10^{12}$; the model was trained on a cluster of $n = 16000$ NVIDIA H100 GPUs (https://arxiv.org/abs/2407.21783). We get:

In [10]:
# model data (Llama 3.1 405b)
N = 405e9
D = 15.6e12

# cluster data (Llama 3.1 405b)
n = 16000  # Number of H100 GPUs in the cluster

# Hardware parameters for NVIDIA H100 GPUs
F_max = 1980e12  # 1980 TFLOPS for NVIDIA H100 in FP16 precision
B = 900e9  # e.g., 900 GB/s for NVLink on H100
M = 80e9  # 80 GB for NVIDIA H100
W = 700 # GPU Watt consumption

# price (purchase and rent) for the NVIDIA H100
c_rent = 6.9e-4
P_gpu = 25e3

In [11]:
# get estimates
training_estim(N, D, F_max, M, B, W, b_max, a_mfu, a_pue, c_j, c_rent, c_adj, P_gpu)

Training time: 35 days
Energy used: 1.21E+07 kWh
Energy cost: 0.97 million $
Renting cost: 33.03 million $
Hardware cost: 800.00 million $


These values are again close to the figures reported in the paper; e.g. the training time was around $60$ days. The discrepancy is due to the fact that the training of Llama 3.1 was not as straightforward as described above, but was done in different stages, where the first stage used half of the available GPUs. 

## Example 3: the cluster of 4090s


An important threat model in AI governance is the possibility of model training using non-specialized or even consumer-grade equipment. To estimate this, we run a hypothetical scenario where one attempts to train a GPT-4 model using a cluster of the most powerful consumer-grade GPUs, the NVIDIA 4090.

For the 4090, we have the following data: $F = 330 \cdot 10^{12}$ FLOP/s (for the `bfloat16` format) and $M = 24 \cdot 10^9$ bytes from the specs (https://www.nvidia.com/en-eu/geforce/graphics-cards/40-series/rtx-4090/). For the bandwidth, technologies such as the NVLink work only with data center GPUs. Here, we’ll assume a high-end standard Ethernet connection of $B = 100Gbit/s = 12.5GB/s$.

First, in order to get the same FLOP/s as the $10000$ H100 cluster, we need the following number of 4090s: $n = 10^4 F_{H100} / F_{4090} = 6 \cdot 10^4$ GPUs. For the training time, we are assuming the same $a_{MFU}$ coefficient (which is fairly optimistic).

For the costs, we have the price $P_{GPU} = 1500 \$$, and assume $c_rent = 0.3 \$/h = 8.310-5 \$/s$. With these, we get:



In [12]:
# model data (GPT-4 with 4090s)
N = 1e12
D = 13e12

# cluster data
n = 60000  # Number of 4090 GPUs in the cluster

# Hardware parameters for NVIDIA 4090
F_max = 330e12  # FP16 precision
B = 100e9 / 8  # e.g., assuming 100Gbit Ethernet
M = 24e9  # 80 GB for NVIDIA 4090
W = 450 # GPU Watt consumption

c_rent = 8.33e-5 # 0.3 $/h
P_gpu = 1.5e3 # 1500$ price

In [13]:
# get estimates
training_estim(N, D, F_max, M, B, W, b_max, a_mfu, a_pue, c_j, c_rent, c_adj, P_gpu)

Training time: 114 days
Energy used: 9.60E+07 kWh
Energy cost: 7.67 million $
Renting cost: 49.22 million $
Hardware cost: 180.00 million $


However, this simple model ignores multiple details that would make a 4090 cluster much harder to use in practice: the difficulty of coordinating that large number of devices without specialized infrastructure and software, the much higher expected failure rates of the devices (which is a substantial challenge in large training runs), the difficulty of building cooling solutions for such a cluster, and many more. Moreover, we see that the overall costs would still be similar as before, even in this simplified setup. 